In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/My\ Drive/Paper_human_loop

Mounted at /content/drive
/content/drive/My Drive/Paper_human_loop


**Load training and test dataset**

load_data(dataset_name, with_features), with_features=True, to use external features

In [2]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder

TitleStyle = ["objective", "subjective", "unknown"]
TitleStance = ["agree", "disagree", "unrelated"]

def load_data(file, with_features):
    df = pd.read_json(file, 'index')
    df = df.rename(columns={"VALUE_reliability": "label", "TITLE": 'text_a', 'TEXT': 'text_b'})
    labels = df['label']
    text_a = df['text_a'].replace(r'\n','', regex=True)

    df['text_b'] = df['text_b'].replace(r'<PARAGRAPH>', '', regex=True)
    df['text_b'] = df['text_b'].replace('<TEXT>', '',  regex=True)
    df['text_b'] = df['text_b'].replace('<BREAK>', ' ',  regex=True)
    df['text_b'] = df['text_b'].replace(r'\n\n\n\n', ' ', regex=True)
    df['text_b'] = df['text_b'].replace(r'\n\n', ' ', regex=True)
    df['text_b'] = df['text_b'].replace(r'\n', ' ', regex=True)
    df['text_b'] = df['text_b'].replace(r'    ', ' ', regex=True)
    df['text_b'] = df['text_b'].replace(r'   ', ' ', regex=True)
    df['text_b'] = df['text_b'].replace(r'  ', ' ', regex=True)
    text_b = df['text_b']


    reduce = [ "TitleStyleObjective",
                "TitleStyleSubjective",
                "TitleStyleUnknown",
                "TitleTitle-StanceAgree",
                "TitleTitle-StanceDisagree",
                "TitleTitle-StanceUnrelated"]

    features = ["Title",
                "Subtitle",
                "Lead",
                "Body",
                "Conclusion",
                "What",
                "WhatReliabilityReliable",
                "WhatReliabilityUnreliable",
                "WhatLack-Of-InformationYes",
                "WhatLack-Of-InformationNo",
                "WhatMain-Event",
                "Who",
                "WhoReliabilityReliable",
                "WhoReliabilityUnreliable",
                "WhoLack-Of-InformationYes",
                "WhoLack-Of-InformationNo",
                "When",
                "WhenReliabilityReliable",
                "WhenReliabilityUnreliable",
                "WhenLack-Of-InformationYes",
                "WhenLack-Of-InformationNo",
                "Where",
                "WhereReliabilityReliable",
                "WhereReliabilityUnreliable",
                "WhereLack-Of-InformationYes",
                "WhereLack-Of-InformationNo",
                "Why",
                "WhyReliabilityReliable",
                "WhyReliabilityUnreliable",
                "WhyLack-Of-InformationYes",
                "WhyLack-Of-InformationNo",
                "How",
                "HowReliabilityReliable",
                "HowReliabilityUnreliable",
                "HowLack-Of-InformationYes",
                "HowLack-Of-InformationNo",
                "Quote",
                "QuoteAuthor-StanceAgree",
                "QuoteAuthor-StanceDisagree",
                "QuoteAuthor-StanceUnknown",
                "WhoRoleSubject",
                "WhoRoleTarget",
                "WhoRoleBoth",
                "Key-Expression",
                "Orthotypography",
                "Figure",
                ]

    df = pd.json_normalize(df['DATA'])
    df_1 = df[reduce]
    conditions_stance = [
        (df_1['TitleTitle-StanceAgree'] == 1),
        (df_1['TitleTitle-StanceDisagree'] == 1),
        (df_1['TitleTitle-StanceUnrelated'] == 1)
    ]
    conditions_style = [
        (df_1['TitleStyleObjective'] == 1),
        (df_1['TitleStyleSubjective'] == 1),
        (df_1['TitleStyleUnknown'] == 1)
    ]

    # create a new column and use np.select to assign values to it using our lists as arguments
    df_1['stance'] = np.select(conditions_stance, TitleStance)
    df_1['style'] = np.select(conditions_style, TitleStyle)
    df_1 = df_1.drop(columns=reduce, axis=1)

    # encode columns stance and style
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(df_1['stance'].values)
    df_1['stance'] = integer_encoded
    integer_encoded = label_encoder.fit_transform(df_1['style'].values)
    df_1['style'] = integer_encoded

    #To labels
    labels = pd.Series(label_encoder.fit_transform(labels.values))
    
  
    #other features
    df_2 = df[features]
    df_2.head()
        
    if with_features: 
      df = pd.concat([text_a, text_b, labels, df_1, df_2], axis=1)
    else:
      df = pd.concat([text_a, text_b, labels], axis=1)    
    return df

df_train = load_data("fake_news_train_reliability_fake.json", True)
df_test = load_data("test_reliability_fake.json", True)


<ipython-input-2-32f5408c8f33>:11: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only
  df = pd.read_json(file, 'index')
<ipython-input-2-32f5408c8f33>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['stance'] = np.select(conditions_stance, TitleStance)
<ipython-input-2-32f5408c8f33>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['style'] = np.select(conditions_style, TitleStyle)


To concatenate title and body text

In [3]:
########################################### Train #######################
df_train.insert(0,"text", list(df_train["text_a"] + " " + df_train["text_b"]))
df_train.head()

########################################### Test ######################
df_test.insert(0,"text", list(df_test["text_a"] + " " + df_test["text_b"]))
df_test.head()



,text,text_a,text_b,0,stance,style,Title,Subtitle,Lead,Body,...,Quote,QuoteAuthor-StanceAgree,QuoteAuthor-StanceDisagree,QuoteAuthor-StanceUnknown,WhoRoleSubject,WhoRoleTarget,WhoRoleBoth,Key-Expression,Orthotypography,Figure
0,Exempleada de Facebook denuncia nulas medidas ...,Exempleada de Facebook denuncia nulas medidas ...,Sabemos que además de ser un buen lugar para ...,1,0,1,1,0,1,1,...,8,0,0,7,10,1,0,1,0,1
1,"Quién es Abhigya Anand, el niño astrólogo de 1...","Quién es Abhigya Anand, el niño astrólogo de 1...","Nacido en la India, alertó al mundo en agosto...",1,0,1,1,1,1,1,...,3,3,0,0,8,2,0,3,0,0
2,Viralizan 'plano' para 'insertar' chips 5G en ...,Viralizan 'plano' para 'insertar' chips 5G en ...,A finales de diciembre empezó a viralizarse e...,0,0,0,1,0,1,1,...,1,0,0,1,1,1,0,0,0,0
3,Julieta Fierro sobre el valor de la astronomía...,Julieta Fierro sobre el valor de la astronomía...,"El 2020 fue un año marcado, además de la pand...",0,0,0,1,0,1,1,...,3,0,0,3,9,1,0,0,0,0
4,La ONU establece uso de chip para identificar ...,La ONU establece uso de chip para identificar ...,"La ONU planea que para el año 2030, cada pers...",1,0,1,1,0,1,1,...,2,2,0,0,7,7,1,0,2,0


Clean text

In [4]:
import re
def de_emojify(text):
    regrex_pattern = re.compile(pattern="["
                                        u"\U0001F600-\U0001F92F"  # emoticons
                                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                        u"\U00002702-\U000027B0"
                                        u"\U000024C2-\U0001F251"
                                        u"\U0001F190-\U0001F1FF"
                                        u"\U0001F926-\U0001FA9F"                                        
                                        u"\u2640-\u2642"
                                        u"\u2600-\u2B55"
                                        u"\u200d"
                                        u"\u23cf"
                                        u"\u23e9"
                                        u"\u231a"
                                        u"\ufe0f"                                        
                                        "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'', text)

def preprocess(value):
    new_value = de_emojify(value)
    new_value = re.sub(r'http\S+', '', new_value)
    return new_value
############################### Train ###############################
df_train["text"] = df_train["text"].str.lower() 
df_train["text"] = df_train.text.apply(preprocess)
df_train.head()
############################### Test ################################
df_test["text"] = df_test["text"].str.lower() 
df_test["text"] = df_test.text.apply(preprocess)
df_test.head()

,text,text_a,text_b,0,stance,style,Title,Subtitle,Lead,Body,...,Quote,QuoteAuthor-StanceAgree,QuoteAuthor-StanceDisagree,QuoteAuthor-StanceUnknown,WhoRoleSubject,WhoRoleTarget,WhoRoleBoth,Key-Expression,Orthotypography,Figure
0,exempleada de facebook denuncia nulas medidas ...,Exempleada de Facebook denuncia nulas medidas ...,Sabemos que además de ser un buen lugar para ...,1,0,1,1,0,1,1,...,8,0,0,7,10,1,0,1,0,1
1,"quién es abhigya anand, el niño astrólogo de 1...","Quién es Abhigya Anand, el niño astrólogo de 1...","Nacido en la India, alertó al mundo en agosto...",1,0,1,1,1,1,1,...,3,3,0,0,8,2,0,3,0,0
2,viralizan 'plano' para 'insertar' chips 5g en ...,Viralizan 'plano' para 'insertar' chips 5G en ...,A finales de diciembre empezó a viralizarse e...,0,0,0,1,0,1,1,...,1,0,0,1,1,1,0,0,0,0
3,julieta fierro sobre el valor de la astronomía...,Julieta Fierro sobre el valor de la astronomía...,"El 2020 fue un año marcado, además de la pand...",0,0,0,1,0,1,1,...,3,0,0,3,9,1,0,0,0,0
4,la onu establece uso de chip para identificar ...,La ONU establece uso de chip para identificar ...,"La ONU planea que para el año 2030, cada pers...",1,0,1,1,0,1,1,...,2,2,0,0,7,7,1,0,2,0


Convert text to tfidf vectors

In [5]:
from sklearn.feature_extraction.text import  TfidfVectorizer
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem.porter import PorterStemmer

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems


def create_tf_idf_model(df):
  # create the transform
  tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=1000, use_idf=True,
                              norm='l2', stop_words=stopwords.words("spanish"), tokenizer=tokenize)

  # tokenize and build vocab
  tfidf.fit(df.text)
  return tfidf

def convert_text_to_features(df, tfidf):  
  vector = tfidf.transform(df.text).todense()

  new_cols = tfidf.get_feature_names()

  # remove the text column as the word 'text' may exist in the words and you'll get an error
  df = df.drop('text_a',axis=1)
  df = df.drop('text_b',axis=1)
  df = df.drop('text',axis=1)
  df.head()

  # join the tfidf values to the existing dataframe
  df = df.join(pd.DataFrame(vector, columns=new_cols))

  return df
tf_idf = create_tf_idf_model(df_train)
df_train = convert_text_to_features(df_train, tf_idf)
df_test = convert_text_to_features(df_test, tf_idf)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['alguna', 'alguno', 'ant', 'desd', 'dond', 'durant', 'ello', 'entr', 'erai', 'ere', 'est', 'estabai', 'estamo', 'estaremo', 'estaréi', 'estaríai', 'estaríamo', 'estemo', 'estuv', 'estuvierai', 'estuvies', 'estuviesei', 'estuvimo', 'estuvist', 'estuvistei', 'estuviéramo', 'estuviésemo', 'estábamo', 'estái', 'estéi', 'fuerai', 'fues', 'fuesei', 'fuimo', 'fuist', 'fuistei', 'fuéramo', 'fuésemo', 'ha

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import precision_score, accuracy_score,f1_score,recall_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import numpy as np
np.random.seed(35)


models = [
          SVC(kernel="linear", C=0.025),           
          RandomForestClassifier(random_state=1, max_depth=5), 
          LogisticRegression(random_state=0),
          DecisionTreeClassifier(max_depth=5),          
          MLPClassifier(alpha=1, max_iter=1000),
          ]
df_train, df_eval = train_test_split(df_train, test_size=0.2, train_size=0.8, random_state=1)
##################### Train #########################
y_train = df_train.iloc[:,0].values
x_train = df_train.iloc[:,1:].values
##################### Dev #########################
y_dev = df_eval.iloc[:,0].values
x_dev = df_eval.iloc[:,1:].values
##################### Test #########################
y_test = df_test.iloc[:,0].values
x_test = df_test.iloc[:,1:].values

for model in models:                
  model.fit(x_train,y_train)
  y_pred = model.predict(x_dev)          
      
  print(accuracy_score(y_dev, y_pred.round()))
  print(f1_score(y_dev, y_pred.round(), average='macro'))

  y_pred = model.predict(x_test)          
      
  print(accuracy_score(y_test, y_pred.round()))
  print(f1_score(y_test, y_pred.round(), average='macro'))



0.9166666666666666
0.916083916083916
0.95
0.9466666666666665
0.9333333333333333
0.9330357142857143
0.88
0.8697916666666666


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9
0.6051136363636364
0.94
0.9356499356499357
0.9
0.898989898989899
0.87
0.8597778017473843
0.9
0.6051136363636364
0.93
0.9244957394024378
